<a href="https://colab.research.google.com/github/aenoboa1/Draft_ML_EmotionAPP/blob/main/Main_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de ML para la deteccion de Emociones


Flujo de trabajo a seguir en este proyecto:

1. Mirando el panorama general.
2. Obtenga los datos.
3. Descubra y visualice los datos para obtener información.
4. Prepare los datos para los algoritmos de Machine Learning.
5. Seleccione un modelo y entrénelo.
6. Ajuste su modelo.
7. Presente su solución.
8. Inicie, supervise y mantenga su sistema 

# 1. Mirando el panorama general.


-- Describir el problema.

# 2. Obtenga los datos.


In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="ml_colab")


In [ ]:
import os
os.chdir("/content/drive/")
!ls

os.chdir("MyDrive/DATA")
!ls


MyDrive  Othercomputers  Shareddrives
'archive (1).zip'   datasets    ngrok
 DAiSEE.zip	   'jupyter '   ngrok-stable-linux-amd64.zip


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   42G  185G  19% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  821M  59% /sbin/docker-init
tmpfs           6.4G   36K  6.4G   1% /var/colab
/dev/sda1       233G   45G  188G  20% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           226G   51G  175G  23% /content/drive


In [ ]:
!unzip DAiSEE.zip -d /home/daisee/

## Data Selection

In [ ]:
os.chdir("/content/drive/MyDrive/DATA/datasets/DAiSEE/Data/")

!ls -l

total 139
drwx------ 2 root root  4096 Dec  7 13:58 Data
-rw------- 1 root root  1123 Apr  6  2018 extractFrames.py
-rw------- 1 root root 27969 Sep 25  2017 Test.txt
-rw------- 1 root root 82082 Apr  6  2018 Train.txt
-rw------- 1 root root 25847 Sep 25  2017 Validation.txt


In [ ]:
!cat extractFrames.py

'''
	-The given code extracts all the frames for the entire dataset and saves these frames in the folder of the video clips.
	-Kindly have ffmpeg (https://www.ffmpeg.org/) (all credits) in order to successfully execute this script.
	-The script must in the a same directory as the Dataset Folder.
'''

import os
import subprocess

dataset = os.listdir('Data/')

def split_video(video_file, image_name_prefix, destination_path):
    return subprocess.check_output('ffmpeg -i "' + destination_path+video_file + '" ' + image_name_prefix + '%d.jpg -hide_banner', shell=True, cwd=destination_path)

for ttv in dataset:
    users = os.listdir('Data/'+ttv+'/')
    for user in users:
        currUser = os.listdir('Data/'+ttv+'/'+user+'/')
        for extract in currUser:
            clip = os.listdir('Data/'+ttv+'/'+user+'/'+extract+'/')[0]
            print (clip[:-4])
            path = os.path.abspath('.')+'/Data/'+ttv+'/'+user+'/'+extract+'/'
            split_video(clip, clip[:-4], path)

print (

In [ ]:
!sudo apt-get install ffmpeg 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


## Extrayendo los Frames del Dataset

In [ ]:
!python extractFrames.py


In [ ]:
import torch
 
torch.cuda.get_device_name(device=None)

'Tesla K80'

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        575M         10G        1.2M        2.0G         11G
Swap:            0B          0B          0B
